In [7]:
% matplotlib inline
import os
import sys
from gensim.models.ldamodel import LdaModel
import gensim
import pandas as pd
import glob
from nltk.corpus import stopwords
import string
from gensim import corpora
import timeit
from collections import defaultdict
from nltk import PorterStemmer
from gensim.utils import lemmatize # Only keep nums
import re
from bs4 import BeautifulSoup
import urllib
import time
import threading
from gensim.corpora import Dictionary


## change dir
os.chdir(os.getcwd()+'\data')

In [8]:
print(sys.version_info) ### Make sure you are using Python3

sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)


In [9]:
dict_data = {}
country_list = []
content_list = []        
for i in glob.glob('*.txt'):
    with open(i,encoding='utf-8',errors = 'ignore') as f:
        country_list += [i.split('.')[0]]
        content = f.readlines()
        content_list += [''.join(list(filter(None, list(map(lambda x:x.strip(), content)))))]
dict_data['country'] = country_list
dict_data['content'] = content_list
df_all = pd.DataFrame(dict_data)

In [10]:
df_all.head()

,content,country
0,IslamicRepublicofAfghanistanMinistryofCommunic...,Afghanistan
1,Shkresa-KokaColorR E P U B L I K A E S H Q I P...,Albania (nonEnglish)
2,AUSTRALIASCYBER SECURITY STRATEGYEnabling inno...,Australia
3,Logo Republik sterreichLogo Bundeskanzleramt s...,Austria
4,Az.rbaycan Prezidentinin R.smi internet s.hif....,Azerbaijan (nonEnglish)


In [11]:
bigram = gensim.models.Phrases(content_list)

In [12]:
def process_text(paragraph_list):
    """
    stopwords
    
    punctuation
    
    digits
    
    
    """
    stoplist = stopwords.words('english') + ['security','cyber','the','information']
    
    remove_digits = str.maketrans('', '', string.digits)
    remove_punctuation = str.maketrans('', '', string.punctuation)
    
    
    paragraph_cleaned =[i.translate(remove_digits) for i in paragraph_list]
    
    paragraph_cleaned = [i.translate(remove_punctuation) for i in paragraph_cleaned]
    
    paragraph_cleaned = [[word.lower() for word in line.split() if word.lower() not in stoplist] for line in paragraph_cleaned]

    return paragraph_cleaned

In [13]:
paragraph_cleaned = process_text(content_list)

## bigram model----input should be inputs from profiles instead

In [14]:
bigram = gensim.models.Phrases(process_text(content_list),min_count=1, threshold=2)

In [15]:
train_text_list = list(bigram[process_text(content_list)])

C:\Users\Miya\Anaconda3\lib\site-packages\gensim\models\phrases.py:248: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


## dictionary built----input should be from profiles instread, modified later

In [16]:
dictionary = Dictionary(train_text_list)
corpus = [dictionary.doc2bow(text) for text in train_text_list]

## lda modeling

In [17]:
lda = LdaModel(corpus, num_topics=10,id2word = dictionary)

[(0,
  '0.003*"national" + 0.002*"de" + 0.002*"development" + 0.002*"systems" + 0.001*"strategy"'),
 (1,
  '0.002*"de" + 0.002*"e" + 0.002*"national" + 0.001*"government" + 0.001*"systems"'),
 (2,
  '0.002*"government" + 0.002*"national" + 0.002*"systems" + 0.001*"strategy" + 0.001*"de"'),
 (3,
  '0.004*"de" + 0.002*"e" + 0.002*"national" + 0.001*"development" + 0.001*"government"'),
 (4,
  '0.002*"national" + 0.002*"de" + 0.001*"development" + 0.001*"systems" + 0.001*"government"'),
 (5,
  '0.002*"national" + 0.002*"ict" + 0.002*"e" + 0.002*"de" + 0.002*"government"'),
 (6,
  '0.002*"national" + 0.002*"government" + 0.002*"systems" + 0.001*"development" + 0.001*"strategy"'),
 (7,
  '0.002*"systems" + 0.002*"national" + 0.002*"development" + 0.002*"e" + 0.002*"government"'),
 (8,
  '0.002*"national" + 0.002*"government" + 0.002*"ict" + 0.002*"systems" + 0.002*"cyberspace"'),
 (9,
  '0.003*"de" + 0.002*"e" + 0.002*"ict" + 0.002*"development" + 0.002*"national"')]

In [19]:
lda.show_topics(100,10)

[(0,
  '0.003*"national" + 0.002*"de" + 0.002*"development" + 0.002*"systems" + 0.001*"strategy" + 0.001*"e" + 0.001*"ensure" + 0.001*"ict" + 0.001*"system" + 0.001*"cyberspace"'),
 (1,
  '0.002*"de" + 0.002*"e" + 0.002*"national" + 0.001*"government" + 0.001*"systems" + 0.001*"strategy" + 0.001*"development" + 0.001*"cyberspace" + 0.001*"services" + 0.001*"also"'),
 (2,
  '0.002*"government" + 0.002*"national" + 0.002*"systems" + 0.001*"strategy" + 0.001*"de" + 0.001*"cyberspace" + 0.001*"cybersecurity" + 0.001*"infrastructure" + 0.001*"well" + 0.001*"also"'),
 (3,
  '0.004*"de" + 0.002*"e" + 0.002*"national" + 0.001*"development" + 0.001*"government" + 0.001*"–" + 0.001*"systems" + 0.001*"infrastructure" + 0.001*"system" + 0.001*"strategy"'),
 (4,
  '0.002*"national" + 0.002*"de" + 0.001*"development" + 0.001*"systems" + 0.001*"government" + 0.001*"cyberspace" + 0.001*"strategy" + 0.001*"policy" + 0.001*"technology" + 0.001*"ict"'),
 (5,
  '0.002*"national" + 0.002*"ict" + 0.002*"e" 

In [22]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 2),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 2),
 (17, 2),
 (18, 7),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 2),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 3),
 (41, 1),
 (42, 1),
 (43, 3),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 2),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 6),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 3),
 (73, 1),
 (74, 1),
 (75, 3),
 (76, 4),
 (77, 1),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 2),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 2),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 2),
 (99, 1),
 (100, 2),